# **Despliegue de Modelos en Azure Machine Learning**


### **1. Introducción al Despliegue de Modelos**
El despliegue de un modelo es el proceso de exponerlo como un servicio web o API RESTful que pueda ser consumido por aplicaciones externas. Azure Machine Learning proporciona varias opciones para desplegar modelos, dependiendo del escenario y los requisitos de producción.

#### **1.1. ¿Por qué es importante el despliegue?**
- Permite integrar modelos de machine learning en aplicaciones empresariales.
- Facilita la automatización de decisiones basadas en predicciones.
- Proporciona escalabilidad para manejar grandes volúmenes de solicitudes.

---

### **2. Opciones de Despliegue en Azure ML**
Azure ML ofrece múltiples opciones para desplegar modelos, cada una diseñada para diferentes necesidades y escenarios.

#### **2.1. Azure Container Instances (ACI)**
- **Descripción:**  
  - Ideal para pruebas rápidas o despliegues pequeños.
  - Se ejecuta en contenedores Docker sin necesidad de configurar infraestructura compleja.
- **Cuándo usarlo:**  
  - Pruebas iniciales del modelo.
  - Escenarios con bajo tráfico o carga reducida.
- **Ventajas:**  
  - Fácil de configurar.
  - Costo bajo para pruebas rápidas.
- **Ejemplo Práctico: Despliegue en ACI**
  ```python
  from azureml.core.webservice import AciWebservice
  from azureml.core.model import InferenceConfig

  inference_config = InferenceConfig(entry_script="score.py")
  deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

  service = Model.deploy(workspace=ws,
                         name="mi-servicio-aci",
                         models=[model],
                         inference_config=inference_config,
                         deployment_config=deployment_config)
  service.wait_for_deployment(show_output=True)
  print(f"Endpoint disponible en: {service.scoring_uri}")
  ```

---

#### **2.2. Azure Kubernetes Service (AKS)**
- **Descripción:**  
  - Diseñado para entornos de producción con alta demanda.
  - Ofrece escalabilidad, balanceo de carga y monitoreo avanzado.
- **Cuándo usarlo:**  
  - Despliegues en producción con alto tráfico.
  - Escenarios que requieren escalabilidad automática.
- **Ventajas:**  
  - Soporte para múltiples réplicas.
  - Integración con herramientas de monitoreo y alertas.
- **Ejemplo Práctico: Despliegue en AKS**
  ```python
  from azureml.core.compute import AksCompute
  from azureml.core.webservice import AksWebservice

  # Crear un clúster AKS (si no existe)
  aks_target = AksCompute(workspace=ws, name="mi-cluster-aks")

  # Configurar el despliegue
  deployment_config = AksWebservice.deploy_configuration(cpu_cores=2, memory_gb=4)

  service = Model.deploy(workspace=ws,
                         name="mi-servicio-aks",
                         models=[model],
                         inference_config=inference_config,
                         deployment_config=deployment_config,
                         deployment_target=aks_target)
  service.wait_for_deployment(show_output=True)
  print(f"Endpoint disponible en: {service.scoring_uri}")
  ```

---

#### **2.3. Azure Functions**
- **Descripción:**  
  - Ideal para despliegues serverless y eventos disparados por HTTP.
  - Ejecuta el modelo solo cuando se recibe una solicitud.
- **Cuándo usarlo:**  
  - Casos de uso con baja frecuencia de solicitudes.
  - Escenarios donde el costo es una preocupación.
- **Ventajas:**  
  - Bajo costo debido al modelo serverless.
  - Respuesta rápida para eventos puntuales.
- **Ejemplo Práctico: Despliegue en Azure Functions**
  ```python
  from azureml.contrib.functions import package
  from azureml.core.model import InferenceConfig

  inference_config = InferenceConfig(entry_script="score.py")
  package = package(ws, [model], inference_config)
  package.wait_for_creation(show_output=True)
  print(f"Paquete Azure Functions disponible en: {package.get_deploy_status()}")
  ```

---

#### **2.4. IoT Edge**
- **Descripción:**  
  - Diseñado para despliegues en dispositivos periféricos.
  - Ejecuta el modelo directamente en dispositivos IoT.
- **Cuándo usarlo:**  
  - Escenarios de edge computing.
  - Aplicaciones que requieren latencia ultrabaja.
- **Ventajas:**  
  - Reducción de la latencia al procesar datos localmente.
  - Ahorro de ancho de banda al evitar enviar datos al cloud.
- **Ejemplo Práctico: Despliegue en IoT Edge**
  ```python
  from azureml.core.model import Model

  model = Model.register(workspace=ws,
                         model_path="model.pkl",
                         model_name="mi-modelo")

  # Generar un paquete para IoT Edge
  package = Model.package(ws, [model], image_name="mi-modelo-iot-edge")
  package.wait_for_creation(show_output=True)
  print(f"Paquete IoT Edge disponible en: {package.get_deploy_status()}")
  ```

---

### **3. Pruebas de API**
Una vez desplegado el modelo, es crucial probar el endpoint REST para asegurarse de que funcione correctamente.

#### **3.1. Herramientas de Prueba**
- **Postman:**  
  - Una herramienta popular para probar APIs RESTful.
  - Permite enviar solicitudes HTTP y visualizar las respuestas.
- **Requests en Python:**  
  - Biblioteca de Python para realizar solicitudes HTTP.

#### **3.2. Ejemplo Práctico: Prueba con Requests**
```python
import requests
import json

url = "http://<endpoint-url>/score"
headers = {"Content-Type": "application/json"}
data = {"data": [[5.1, 3.5, 1.4, 0.2]]}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())
```

---

### **4. Ejercicio Práctico**

**Título:** Desplegar un modelo de clasificación en Azure ML y probar su endpoint REST.

#### **Pasos:**
1. **Entrenamiento del Modelo:**
   - Entrena un modelo de clasificación utilizando Scikit-learn.
   ```python
   from sklearn.datasets import load_iris
   from sklearn.linear_model import LogisticRegression
   import joblib

   X, y = load_iris(return_X_y=True)
   model = LogisticRegression()
   model.fit(X, y)
   joblib.dump(model, "model.pkl")
   ```

2. **Despliegue en ACI:**
   - Despliega el modelo en Azure Container Instances.
   ```python
   from azureml.core.webservice import AciWebservice
   from azureml.core.model import InferenceConfig

   inference_config = InferenceConfig(entry_script="score.py")
   deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

   service = Model.deploy(workspace=ws,
                          name="mi-servicio-aci",
                          models=["model.pkl"],
                          inference_config=inference_config,
                          deployment_config=deployment_config)
   service.wait_for_deployment(show_output=True)
   print(f"Endpoint disponible en: {service.scoring_uri}")
   ```

3. **Prueba del Endpoint:**
   - Usa Postman o `requests` para probar el endpoint.
   ```python
   import requests
   import json

   url = "http://<endpoint-url>/score"
   headers = {"Content-Type": "application/json"}
   data = {"data": [[5.1, 3.5, 1.4, 0.2]]}

   response = requests.post(url, headers=headers, data=json.dumps(data))
   print(response.json())
   ```

---

### **Conclusión**
Este módulo proporciona una visión completa de las opciones de despliegue en Azure ML, desde pruebas rápidas con ACI hasta despliegues robustos con AKS. Los participantes estarán preparados para elegir la opción adecuada según sus necesidades y desplegar modelos de manera efectiva.

#### **Referencias Generales:**
- [Azure ML Deployment Documentation](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)
- [Postman Documentation](https://learning.postman.com/docs/getting-started/introduction/)
- [Scikit-learn Documentation](https://scikit-learn.org/stable/documentation.html)